In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

c:\Users\Beeface\.virtualenvs\BookChatbot-9Q2uoHrH\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("Model_Tokenizer/qwen3_0.6_tokenizer")
model = AutoModelForCausalLM.from_pretrained(
    "Model_Tokenizer/qwen3_0.6_model",
    torch_dtype="auto",
    device_map="auto"
)

In [3]:


# print(model.eval())

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>
Okay, the user is asking for a short introduction to a large language model. Let me start by recalling what a large language model is. It's a type of machine learning model designed to understand and generate human language. But I should make sure to explain it clearly without getting too technical.

First, I need to mention that it's a powerful tool that can understand and create text. Maybe start with the key points: it's based on deep learning, can understand language, and can generate text. But I should keep it concise. Also, maybe include an example of its use, like answering questions or writing articles, to show its practical application.

Wait, the user might be looking for a brief but informative intro. I should avoid jargon and keep it simple. Let me check if there's any important detail I'm missing. Oh, right, the model can learn from a large dataset, which is a key feature. Also, it's used in various fields like AI, customer service, etc. 

I shoul

In [3]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, DirectoryLoader, UnstructuredWordDocumentLoader # Loads text documents.
from langchain_community.vectorstores import Chroma, Weaviate # A vector store for efficiently storing and searching vector embeddings.
from langchain_community.embeddings import SentenceTransformerEmbeddings # Converts text chunks into vector embeddings using a pre-trained model.
from langchain.text_splitter import RecursiveCharacterTextSplitter # Splits large text into smaller, manageable chunks.
import os # Used for operating system interactions (though not used explicitly in this script).
import shutil
# from langchain_huggingface import HuggingFaceEmbeddings

# from weaviate.classes.init import Auth
# from weaviate.classes.config import Configure

from dotenv import load_dotenv

load_dotenv()

YOUR_WEAVIATE_KEY = os.environ["YOUR_WEAVIATE_KEY"]
YOUR_WEAVIATE_CLUSTER = os.environ["YOUR_WEAVIATE_CLUSTER"]
YOUR_HUGGINGFACE_KEY = os.environ["YOUR_HUGGINGFACE_KEY"]

In [13]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [WinError 10060] A connection attempt failed because
[nltk_data]     the connected party did not properly respond after a
[nltk_data]     period of time, or established connection failed
[nltk_data]     because connected host has failed to respond>


False

In [ ]:
# loader = DirectoryLoader("PDF", glob="**/*.pdf")  # :contentReference[oaicite:17]{index=17}
# docs = loader.load()
# print(f"You have {len(docs)} documents loaded.")
# print(f"You have {len(docs[0].page_content)} characters in your documents")

Error loading file PDF\Agents_Companion_v2.pdf


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [4]:
pdf_folder = "PDF"  # Folder containing your PDF files
all_docs = []

for filename in os.listdir(pdf_folder):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)
        print(f"Loading: {pdf_path}")
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()
        all_docs.extend(docs)  # Append the loaded pages/documents to the full list

print(f"You have {len(all_docs)} documents loaded.")
print(f"You have {len(all_docs[0].page_content)} characters in your documents")

Loading: PDF\Agents_Companion_v2.pdf
Loading: PDF\Agents_v8.pdf
Loading: PDF\MLOps_Operationalizing Generative AI on Vertex AI_v7 (1).pdf
Loading: PDF\Prompt Engineering_v7 (1).pdf
Loading: PDF\Solving Domain-Specific problems using LLMs_v7.pdf
Loading: PDF\whitepaper_emebddings_vectorstores_v2.pdf
Loading: PDF\whitepaper_Foundational Large Language models & text generation_v2.pdf
You have 466 documents loaded.
You have 195 characters in your documents


In [16]:
loader = PyPDFLoader("PDF/Agents_Companion_v2.pdf")  # :contentReference[oaicite:17]{index=17}
docs = loader.load()
print(f"You have {len(docs)} documents loaded.")
print(f"You have {len(docs[0].page_content)} characters in your documents")

You have 76 documents loaded.
You have 195 characters in your documents


In [22]:
all_docs

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 20.2 (Macintosh)', 'creationdate': '2025-03-18T10:48:11-06:00', 'moddate': '2025-03-18T10:48:21-06:00', 'trapped': '/False', 'source': 'PDF\\Agents_Companion_v2.pdf', 'total_pages': 76, 'page': 0, 'page_label': '1'}, page_content='Agents Companion \n \nAuthors: Antonio Gulli, Lavi Nigam,  \nJulia Wiesinger, Vladimir Vuskovic,  \nIrina Sigler, Ivan Nardini, Nicolas Stroppa, \nSokratis Kartakis, Narek Saribekyan,  \nand Alan Bount'),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 20.2 (Macintosh)', 'creationdate': '2025-03-18T10:48:11-06:00', 'moddate': '2025-03-18T10:48:21-06:00', 'trapped': '/False', 'source': 'PDF\\Agents_Companion_v2.pdf', 'total_pages': 76, 'page': 1, 'page_label': '2'}, page_content='Agents Companion\nFebruary 2025\n2\nAcknowledgements\nEditors & curators\nAnant Nawalgaria\nContent contributors\nAnant Nawalgaria\nSteven Johnson\nHussain Chinoy 

#### Connecting to Weaviate Client

In [52]:
headers = {
    "X-HuggingFace-Api-Key": YOUR_HUGGINGFACE_KEY,
    }

client = weaviate.connect_to_weaviate_cloud(
        cluster_url=YOUR_WEAVIATE_CLUSTER,                       # `weaviate_url`: your Weaviate URL
        auth_credentials=Auth.api_key(YOUR_WEAVIATE_KEY),      # `weaviate_key`: your Weaviate API key
        headers=headers
    )

print(client.is_ready())

C:\Users\Beeface\AppData\Local\Temp\ipykernel_6004\600448803.py:5: ResourceWarning: unclosed <ssl.SSLSocket fd=6976, family=2, type=1, proto=0, laddr=('192.168.1.159', 55325), raddr=('34.36.172.118', 443)>
  client = weaviate.connect_to_weaviate_cloud(


True


In [42]:
client.collections.create(
    "BookChatbot",
    vectorizer_config=[
        Configure.NamedVectors.text2vec_huggingface(
            name="title_vector",
            source_properties=["title"],
            model="sentence-transformers/all-MiniLM-L6-v2",
        ),
        Configure.NamedVectors.text2vec_huggingface(
            name="content_vector",
            source_properties=["content"],
            model="sentence-transformers/all-MiniLM-L6-v2",
        )
    ],
    )

c:\Users\Beeface\.virtualenvs\BookChatbot-9Q2uoHrH\Lib\site-packages\weaviate\collections\classes\config.py:1950: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for cls_field in self.model_fields:


In [45]:
collection = client.collections.get("BookChatbot")
print(collection)

<weaviate.Collection config={
  "name": "BookChatbot",
  "description": null,
  "generative_config": null,
  "inverted_index_config": {
    "bm25": {
      "b": 0.75,
      "k1": 1.2
    },
    "cleanup_interval_seconds": 60,
    "index_null_state": false,
    "index_property_length": false,
    "index_timestamps": false,
    "stopwords": {
      "preset": "en",
      "additions": null,
      "removals": null
    }
  },
  "multi_tenancy_config": {
    "enabled": false,
    "auto_tenant_creation": false,
    "auto_tenant_activation": false
  },
  "properties": [],
  "references": [],
  "replication_config": {
    "factor": 1,
    "async_enabled": false,
    "deletion_strategy": "NoAutomatedResolution"
  },
  "reranker_config": null,
  "sharding_config": {
    "virtual_per_physical": 128,
    "desired_count": 1,
    "actual_count": 1,
    "desired_virtual_count": 128,
    "actual_virtual_count": 128,
    "key": "_id",
    "strategy": "hash",
    "function": "murmur3"
  },
  "vector_index

In [47]:
text_meta_pair = [(doc.page_content, doc.metadata) for doc in all_docs]
texts, meta = list(zip(*text_meta_pair))
print(f"You have {len(texts)} texts and {len(meta)} metadata pairs.")
print(texts)
print(meta)

You have 466 texts and 466 metadata pairs.
('Agents Companion \n \nAuthors: Antonio Gulli, Lavi Nigam,  \nJulia Wiesinger, Vladimir Vuskovic,  \nIrina Sigler, Ivan Nardini, Nicolas Stroppa, \nSokratis Kartakis, Narek Saribekyan,  \nand Alan Bount', 'Agents Companion\nFebruary 2025\n2\nAcknowledgements\nEditors & curators\nAnant Nawalgaria\nContent contributors\nAnant Nawalgaria\nSteven Johnson\nHussain Chinoy \nDesigner\nMichael Lanning', 'Introduction 6\nAgent Ops 8\nAgent Success Metrics 12\nAgent Evaluation 14\nAssessing Agent Capabilities 15\nEvaluating Trajectory and Tool Use 17\nEvaluating the Final Response 20\nHuman-in-the-Loop Evaluation 21\nMore about Agent Evaluation 22\nMultiple Agents & Their Evaluation 23\nUnderstanding Multi-Agent Architectures 24\nMulti-Agent Design Patterns and Their Business Impact 25\nImportant components of Agents 28\nChallenges in Multi-Agent systems  31\nMulti-Agent Evaluation 32\nTable of contents', 'Agentic RAG: A Critical Evolution in Retrieval

In [50]:
with collection.batch.fixed_size(batch_size=100) as batch:
    for text, metadata in zip(texts, meta):
        properties = {
            "content":text,
            **metadata,
        }

        batch.add_object(properties=properties)

{'message': 'Failed to send all objects in a batch of 100', 'error': "WeaviateInsertManyAllFailedError('Every object failed during insertion. Here is the set of all errors: failed with status: 401 error: Invalid username or password.\\nfailed with status: 401 error: Invalid username or password.')"}
{'message': 'Failed to send 100 objects in a batch of 100. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}
{'message': 'Failed to send all objects in a batch of 100', 'error': "WeaviateInsertManyAllFailedError('Every object failed during insertion. Here is the set of all errors: failed with status: 401 error: Invalid username or password.\\nfailed with status: 401 error: Invalid username or password.')"}
{'message': 'Failed to send 100 objects in a batch of 100. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}
{'message': 'Failed to send all objects in a batch of 100', 'error': "We

In [ ]:










client.close()

{'message': 'Failed to send all objects in a batch of 100', 'error': "WeaviateInsertManyAllFailedError('Every object failed during insertion. Here is the set of all errors: failed with status: 401 error: Invalid username or password.')"}
{'message': 'Failed to send 100 objects in a batch of 100. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}
{'message': 'Failed to send all objects in a batch of 100', 'error': "WeaviateInsertManyAllFailedError('Every object failed during insertion. Here is the set of all errors: failed with status: 401 error: Invalid username or password.')"}
{'message': 'Failed to send 100 objects in a batch of 100. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}
{'message': 'Failed to send all objects in a batch of 100', 'error': "WeaviateInsertManyAllFailedError('Every object failed during insertion. Here is the set of all errors: failed with status: 401 e

In [ ]:
query = "What are the main topics discussed in the book?"
doc = collection.similiarity_search(query, top_k=20)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(all_docs)

In [6]:
vector_db_path = "vector_db_path"
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
# embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectordb = Chroma.from_documents(
            documents=splits,
            embedding=embedding_model,
            persist_directory=vector_db_path,
        )

C:\Users\Beeface\AppData\Local\Temp\ipykernel_17864\1787768.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\Beeface\.virtualenvs\BookChatbot-9Q2uoHrH\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
